In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rc
import matplotlib.ticker as ticker
from matplotlib import rc
import scipy.integrate as integrate
from scipy.special import factorial
from tqdm import tqdm

In [ ]:
# Matplotlib customize
plt.rcParams['figure.figsize'] = [6,5]
plt.rcParams['figure.dpi'] = 100

plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 20

plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['lines.markersize'] = 4

plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.markerscale'] = 1.5
plt.rcParams['legend.borderpad'] = 0.6
plt.rcParams['legend.framealpha'] = 0.7

In [ ]:
# Reading channels
data = pd.read_csv('datos.csv',sep=',')
n = np.array(data.n)
b = np.array(data.b)
s = np.array(data.s)
N = n.shape[0]
data

In [ ]:
# Estimator definition
def GetLnQ(n,b,s,mu):
    return 2*(mu*s - n*np.log( 1 + (mu*s/b) ))

In [ ]:
def GetJointLnQ(LnQ,mu,Null=True):

    Q = 0.

    for c in range(N):

        if Null:
            ntoy = np.random.poisson(b[c])
        else:
            ntoy = np.random.poisson( mu*s[c] + b[c] )

        Q += LnQ(ntoy,b[c],s[c],mu)

    return Q

In [ ]:
def GetObsJointLnQ(LnQ,mu,Expected = True):

    Q = 0.
    # The observed LnQ is defined for mu == 1!!!!
    
    for c in range(N):

        if Expected: # Observed events are the expected background
            Q += LnQ( b[c], b[c], s[c], mu )
        else:
            Q += LnQ( n[c], b[c], s[c], mu )

    return Q

In [ ]:
GetJointLnQ(GetLnQ,0.2)

In [ ]:
# The observed LnQ
Qobs = GetObsJointLnQ(GetLnQ,1.0,Expected=True)
Qobs

In [ ]:
def Sampler(mu, Ntoys = int(2e4)):

    q0 = np.zeros(Ntoys)
    q1 = np.zeros(Ntoys)

    for i in range(Ntoys):
        q0[i] = GetJointLnQ(GetLnQ, mu)  #H0
        q1[i] = GetJointLnQ(GetLnQ, mu, Null = False) #H1

    
    return q0,q1

In [ ]:
q0,q1 =  Sampler(1.3)

In [ ]:
plt.hist(q0)
plt.hist(q1)
plt.axvline(x=Qobs,color='k')
#plt.yscale('log')

In [ ]:
def GetPValue(data,Qobs,Null = True):

    if Null:
        count_below_threshold = np.sum( data <= Qobs )
        p_value = count_below_threshold/data.shape[0]

    else:
        count_above_threshold = np.sum( data >= Qobs )
        p_value = count_above_threshold / data.shape[0]

    return p_value

In [ ]:
GetPValue(q1,Qobs,Null = False)

In [ ]:
GetPValue(q0,Qobs,Null = True)

In [ ]:
mu = np.linspace(1.0,4.0,10)

In [ ]:
# p-value scan
def GetCls(mu,Expected=True):

    p_value = np.zeros_like(mu)

    for i in tqdm(range(mu.shape[0])):

        Qobs = GetObsJointLnQ(GetLnQ,1.0,Expected)
        q0,q1 = Sampler(mu[i])

        p0 = GetPValue(q0,Qobs,Null = True)
        p1 = GetPValue(q1,Qobs,Null = False)

        if p0 == 1:
            print('Problema con este valor:', mu[i])
            pmu = p1 
        else:
            pmu = p1 / ( 1. - p0)

        p_value[i] = pmu

    return p_value

In [ ]:
p_value = GetCls(mu)

In [ ]:
plt.plot(mu,p_value)
plt.axhline(y=0.05,ls='--',color='r')